Extend network x with MILP flow optimization
- overwrite add note method
    - name
    - demand
    - x, y
- overwrite add edge method
    - origin Node
    - destination Node
    - variable cost
    - fixed cost
    - flow
- create flow graph class inheriting from DiGraph
- attach flow and arc selection variables to Edges
- define constraints
- solve and attache flows to Edges
- visualize flows with plotly


In [1]:
from networkx import DiGraph
import networkx as nx
from pydantic import BaseModel, model_validator
from typing import Optional
from pulp import LpProblem, LpVariable, LpMinimize, LpStatus
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

In [2]:
class FlowViz():

    def __init__(self, graph):
        self.graph = graph            

    def plot_flowgraph(self, edge_width_field='flow', edge_color_field='utilisation', colormap_name='RdYlGn_r', 
                   node_size_type='max_flow', min_width=0.25, max_width=5):
        # set plot properties
        self.edge_width_field = edge_width_field
        self.edge_color_field = edge_color_field
        self.colormap_name = colormap_name
        self.node_size_type = node_size_type
        self.min_width = min_width
        self.max_width = max_width
        
        # get nodes and edges
        self.node_df = self.graph.nodes_to_df().set_index('name')
        self.edge_df = self.graph.edges_to_df().set_index(['origin', 'destination'])

        # set edge and node colores
        self.edge_colormap = plt.get_cmap(self.colormap_name)
        self.node_colors = self._get_color_dict(self.node_df['type'])

        # set sizes
        self.xy_range = self.node_df[['x', 'y']].apply([min, max]).diff().loc['max'].min()       
        if node_size_type == 'max_flow': 
            node_size = self.node_df[['inflow', 'outflow']].max(axis=1)
            self.node_size_range = node_size.apply([min, max])
        elif node_size_type in self.node_df.columns:
            self.node_size_range = self.node_df[node_size_type].apply([min, max])
        else: 
            self.node_size_range = pd.Series({'min': 0, 'max': 1})
        self.width_range = self.edge_df[self.edge_width_field].apply([min, max])

        # create nodes
        node_traces = []
        node_shapes = []
        annotations = []
        for node_name, node in self.node_df.iterrows():
            shape, trace, annotation = self._create_node_object(node_name, node)
            node_traces.append(trace)
            node_shapes.append(shape)
            annotations.append(annotation)        
        
        # create edges
        edge_traces = []
        for (origin_name, destination_name), edge in self.edge_df.iterrows():
            origin = self.node_df.loc[origin_name]
            destination = self.node_df.loc[destination_name]
            edge_traces.extend(self._create_edge_object(edge, origin, destination))

        # create colorbar
        colorbar_trace = self._create_colorbar_object()

        # create legend
        legend_traces = self._create_legend_object()

        # create figure
        fig = go.Figure()
        fig.update_layout(hovermode='closest', 
            xaxis=dict(visible=False),  
            yaxis=dict(visible=False), 
            plot_bgcolor='rgba(0,0,0,0)',  
            paper_bgcolor='rgba(0,0,0,0)', 
            legend=dict(
                x=self.node_df['x'].min() - self.xy_range * 0.15,  
                y=self.node_df['y'].max() + self.xy_range * 0.15,
                traceorder='normal',
                font=dict(size=12, color='white'),
                bordercolor='rgba(0, 0, 0, 0.5)',
                borderwidth=1), 
            shapes=node_shapes, 
            annotations=annotations)   
        fig.add_traces(node_traces + edge_traces + legend_traces)
        # check if colorbar shall be added
        if colorbar_trace: fig.add_traces(colorbar_trace)
        fig.update_yaxes(scaleanchor="x", scaleratio=1)

        return fig
    
    def plot_graph(self):
        return self.plot_flowgraph(edge_width_field='capacity', node_size_type=None, edge_color_field=None)

    @staticmethod
    def _get_field_hover_text(series):
        text = ''
        for key, value in series.items():
            if value is None or value == float('inf'): continue

            # format value
            if isinstance(value, float):
                if abs(value) >= 100:
                    value_str = f"{value:.0f}"
                elif abs(value) >= 10:
                    value_str = f"{value:.1f}"
                else:
                    value_str = f"{value:.2f}"
            elif isinstance(value, int):
                value_str = f"{value}"
            else:
                value_str = value
            
            # format key
            key = key.replace('_', ' ').title()

            # append to text
            text += f"{key}: {value_str}<br>"
        return text        
    
    @staticmethod
    def _get_edge_hover_text(edge):
        title = f"<b>{edge.name[0]} -> {edge.name[1]}</b><br>"
        return title + FlowViz._get_field_hover_text(edge)

    @staticmethod
    def _get_node_hover_text(node_name, node):
        title = f"<b>{node_name}</b><br>"
        return title + FlowViz._get_field_hover_text(node)

    @staticmethod
    def _create_vector(x1, y1, x2, y2, l):
        # create vector along (x1, y1) to (x2, y2) with length l
        dx = x2 - x1
        dy = y2 - y1
        length = (dx**2 + dy**2)**0.5

        if length == 0: return 0, 0
        
        return dx / length * l, dy / length * l 

    @staticmethod    
    def _norm_vals(vals, out_min, out_max):
        return out_min + (out_max - out_min) * (vals - vals.min()) / (vals.max() - vals.min())

    def _get_edge_color(self, val):
        color = self.edge_colormap(val)
        return f'rgba({color[0]*255}, {color[1]*255}, {color[2]*255}, {color[3]})'

    @staticmethod
    def _get_color_dict(names):
        # get unique names and sort
        unique_names = sorted(set(names))
        return {name: DEFAULT_PLOTLY_COLORS[i % len(DEFAULT_PLOTLY_COLORS)] for i, name in enumerate(unique_names)}

    def _get_node_radius(self, node):
        if self.node_size_type == 'max_flow':
            value = max(node['inflow'], node['outflow'])
        elif self.node_size_type in node:
            value = node[self.node_size_type]
        else:
            value = 0.5
        out_min = 0.005 * self.xy_range
        out_max = 0.05 * self.xy_range
        ratio = (value - self.node_size_range['min']) / (self.node_size_range['max'] - self.node_size_range['min'])
        return out_min + (out_max - out_min) * ratio
    
    def _get_edge_width(self, edge):
        ratio = (edge[self.edge_width_field] - self.width_range['min']) / (self.width_range['max'] - self.width_range['min'])
        return self.min_width + (self.max_width - self.min_width) * ratio

    def _create_node_object(self, node_name, node):
        radius = self._get_node_radius(node)
        color = self.node_colors[node['type']]
        text_offset = 0.02 * self.xy_range
        
        shape = go.layout.Shape(type="circle",
                                xref="x", yref="y",
                                fillcolor=self.node_colors[node['type']],
                                x0=node['x'] - radius, y0=node['y'] - radius, 
                                x1=node['x'] + radius, y1=node['y'] + radius,
                                line=dict(color='white', width=1), 
                                layer='below')
        
        marker = go.Scatter(x=[node['x']], y=[node['y']],         
                mode='markers',
                hoverinfo='text',
                hovertext=self._get_node_hover_text(node_name, node),
                marker=dict(size=0, color=color),
                showlegend=False)
        
        text = go.layout.Annotation(x=node['x'], 
                                    y=node['y'] + radius + text_offset,
                                    xref='x',
                                    yref='y',
                                    text=node_name, 
                                    font=dict(color='white'),
                                    bgcolor='black',
                                    showarrow=False, 
                                    opacity=0.8)

        return shape, marker, text
    
    def _create_edge_object(self, edge, origin, destination):    
        if 'flow' in edge and edge['flow'] > 0:
            line_type = 'solid'            
            color = self._get_edge_color(edge[self.edge_color_field]) if self.edge_color_field else 'gray'
        else:
            line_type = 'dash'
            color = 'gray'

        # get radius of origin and destination nodes
        origin_radius = self._get_node_radius(origin)
        destination_radius = self._get_node_radius(destination)

        # create vector from origin to edge of origin node along edge
        origin_radius_x, origin_radius_y = self._create_vector(origin['x'], origin['y'], destination['x'], 
                                                               destination['y'], origin_radius)
        destination_radius_x, destination_radius_y = self._create_vector(destination['x'], destination['y'], origin['x'], 
                                                                         origin['y'], destination_radius)
        # create vector from origin edge to destination edge
        origin_edge_x = origin['x'] + origin_radius_x
        origin_edge_y = origin['y'] + origin_radius_y
        destination_edge_x = destination['x'] + destination_radius_x
        destination_edge_y = destination['y'] + destination_radius_y

        width = self._get_edge_width(edge)

        arrow_trace = go.Scatter(        
            x=[origin_edge_x, destination_edge_x],
            y=[origin_edge_y, destination_edge_y],
            line=dict(width=width, color=color, dash=line_type),
            marker = dict(size=max(width*4, 5), symbol= "arrow-bar-up", angleref="previous", color=color),                            
            mode='lines+markers', 
            hoverinfo='skip', 
            showlegend=False)
        
        edge_trace = go.Scatter(
            x=np.linspace(origin_edge_x, destination_edge_x, 20),
            y=np.linspace(origin_edge_y, destination_edge_y, 20),

            line=dict(width=0, color=color, dash=line_type),
            hoverinfo='text',
            hovertext=self._get_edge_hover_text(edge),
            mode='lines', 
            showlegend=False)
                    
        return [arrow_trace, edge_trace]
    

    def _create_colorbar_object(self):
        if not self.edge_color_field: return None
        colorbar_trace = go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(
                colorscale=self.colormap_name,
                cmin=self.edge_df[self.edge_color_field].min(),
                cmax=self.edge_df[self.edge_color_field].max(),
                colorbar=dict(
                    title='Edge ' + self.edge_color_field.replace('_', ' ').title(),
                    titleside='right', 
                    titlefont=dict(color='white'),  # Set colorbar title font color to white
                    tickfont=dict(color='white'))    # Set colorbar tick font color to white        
            ),
            hoverinfo='none', 
            showlegend=False
        )
        return colorbar_trace
    
    def _create_legend_object(self):
        legend_traces = []
        for name, color in self.node_colors.items():
            legend_trace = go.Scatter(
                x=[None], y=[None],
                mode='markers',
                marker=dict(size=10, color=color, line=dict(width=2)),
                name=name,
                showlegend=True
            )
            legend_traces.append(legend_trace)    
        return legend_traces
    

In [3]:
class FlowGraph(DiGraph):

    # node and edge definitions
    class Node(BaseModel, frozen=True):
        name: str
        demand: Optional[float] = 0.0
        supply: Optional[float] = 0.0
        capacity: Optional[float] = float('inf')
        type: Optional[str] = None
        x: Optional[float] = 0.0
        y: Optional[float] = 0.0
        variable_cost: Optional[float] = 0.0
        fixed_cost: Optional[float] = 0.0

        @model_validator(mode='after')
        def validate(self):
            # check that demand is non-negative
            if self.demand < 0 or self.demand == float('inf'): raise ValueError('demand must be non-negative and finite')
            # check that supply is non-negative
            if self.supply < 0: raise ValueError('supply must be non-negative')
            # check that capacity is non-negative
            if self.capacity < 0: raise ValueError('capacity must be non-negative')
            # check that variable_cost is non-negative
            if self.variable_cost < 0: raise ValueError('variable_cost must be non-negative')
            # check that fixed_cost is non-negative
            if self.fixed_cost < 0: raise ValueError('fixed_cost must be non-negative')
            return self
  
    class Edge(BaseModel, frozen=True):
        origin: 'FlowGraph.Node'
        destination: 'FlowGraph.Node'
        capacity: Optional[float] = float('inf')
        variable_cost: Optional[float] = 0.0
        fixed_cost: Optional[float] = 0.0
        
        @model_validator(mode='after')
        def validate(self):
            # check that node names are different
            if self.origin.name == self.destination.name: raise ValueError('origin and destination names must be different')
            # check that capacity is non-negative
            if self.capacity < 0: raise ValueError('capacity must be non-negative')
            # check that variable_cost is non-negative
            if self.variable_cost < 0: raise ValueError('variable_cost must be non-negative')
            # check that fixed_cost is non-negative
            if self.fixed_cost < 0: raise ValueError('fixed_cost must be non-negative')
            return self

    def __init__(self, incoming_graph_data=None, **attr):
        self.soft_constr = False
        # check what input data is given
        if isinstance(incoming_graph_data, FlowGraph): 
            # get all nodes and edges
            graph_elements = incoming_graph_data.get_nodes() + incoming_graph_data.get_edges()
        elif isinstance(incoming_graph_data, list): 
            graph_elements = incoming_graph_data
        elif incoming_graph_data is None: 
            graph_elements = []
        else:
            raise ValueError('incoming_graph_data must be a list, FlowGraph object or None')
        
        # initialialize digraph
        super().__init__(None, **attr)
        # add nodes and edges
        for element in graph_elements:
            if isinstance(element, FlowGraph.Node): self.add_node(element)
            elif isinstance(element, FlowGraph.Edge): self.add_edge(element)
            else: raise ValueError('incoming_graph_data must be a list of Node elements or Edge elements')

        # initialize visualization object
        self.viz = FlowViz(self)


    def add_node(self, node):
        # check if node is a Node element
        if not isinstance(node, FlowGraph.Node): raise ValueError('node must be a Node element')
        # add node to graph
        super().add_node(node.name, demand=node.demand, supply=node.supply, capacity=node.capacity, type=node.type, 
                         variable_cost=node.variable_cost, fixed_cost=node.fixed_cost, x=node.x, y=node.y)

    def add_nodes_from(self, nodes):
        # check if nodes are dataframe
        if isinstance(nodes, pd.DataFrame):
            # get columns to create nodes
            col_select = [col for col in nodes.columns if col in FlowGraph.Node.__fields__.keys()]
            for _, node in nodes[col_select].iterrows():                
                self.add_node(FlowGraph.Node(**node.to_dict()))
        elif nodes: 
            for node in nodes: self.add_node(node)


    def add_edge(self, edge):       
        # check if edge is an Edge element
        if not isinstance(edge, FlowGraph.Edge): raise ValueError('edge must be an Edge element')
        # check if nodes exist
        if not edge.origin.name in super().nodes: self.add_node(edge.origin)
        if not edge.destination.name in super().nodes: self.add_node(edge.destination)

        # add edge to graph
        super().add_edge(edge.origin.name, edge.destination.name, capacity=edge.capacity, 
                         variable_cost=edge.variable_cost, fixed_cost=edge.fixed_cost)


    def add_edges_from(self, edges):
        # check if edges are dataframe
        if isinstance(edges, pd.DataFrame):
            # get columns to create edges
            col_select = [col for col in edges.columns if col in FlowGraph.Edge.__fields__.keys()]
            for _, edge in edges[col_select].iterrows():
                edge_dict = edge.to_dict()
                # get nodes
                origin = self.get_node(edge_dict.pop('origin'))
                destination = self.get_node(edge_dict.pop('destination'))
                self.add_edge(FlowGraph.Edge(origin=origin, destination=destination, **edge_dict))
        elif edges: 
            for edge in edges: self.add_edge(edge)


    def add_weighted_edges_from(self, edges):        
        self.add_edges_from(edges)


    def get_node(self, name):
        if name in super().nodes:
            data = super().nodes[name]
            return FlowGraph.Node(name=name, demand=data['demand'], supply=data['supply'], capacity=data['capacity'], 
                                  type=data['type'], variable_cost=data['variable_cost'], fixed_cost=data['fixed_cost'], 
                                  x=data['x'], y=data['y'])
        else:
            raise ValueError(f'node {name} not found')
    

    def get_nodes(self):
        return [self.get_node(name) for name in super().nodes]
    

    def get_edge(self, origin_name, destination_name):
        if origin_name in super().nodes and destination_name in super().nodes:
            data = super().edges[origin_name, destination_name]
            return FlowGraph.Edge(origin=self.get_node(origin_name), destination=self.get_node(destination_name), 
                                  capacity=data['capacity'], variable_cost=data['variable_cost'], 
                                  fixed_cost=data['fixed_cost'])
        else:
            raise ValueError(f'edge {origin_name}-{destination_name} not found')


    def get_edges(self):
        return [self.get_edge(origin_name, destination_name) for origin_name, destination_name in super().edges]
    

    def _assign_decision_variables(self):
        # variable counts
        cont_count = bin_count = 0
        # assign decision variables to all edges
        for origin_name, destination_name, edge in super().edges.data():
            # create flow variable if capacity is positive or soft constraints are used
            edge['flow_var'] = (LpVariable(f"flow_{origin_name}-{destination_name}", lowBound=0, cat='Continuous') 
                                if edge['capacity'] > 0 or self.soft_constr else None)            
            if edge['flow_var'] is not None: cont_count += 1

            # check if capacity variable is needed
            if self.soft_constr and edge['capacity'] < float('inf'):
                edge['capacity_var'] = LpVariable(f"capacity_{origin_name}-{destination_name}", lowBound=0, 
                                                  cat='Continuous')
                cont_count += 1
            else:
                edge['capacity_var'] = None

            edge['selection_var'] = (LpVariable(f"selection_{origin_name}-{destination_name}", cat='Binary') 
                                     if edge['capacity'] > 0 and edge['fixed_cost'] > 0 else None)
            if edge['selection_var'] is not None: bin_count += 1

        # assign decision variables to all nodes
        for name, node in super().nodes.data():
            node['selection_var'] = LpVariable(f"selection_{name}", cat='Binary') if node['fixed_cost'] > 0 else None
            if node['selection_var'] is not None: bin_count += 1

            # check if supply variable is needed
            if self.soft_constr and node['supply'] > node['demand'] and node['supply'] < float('inf'):
                node['supply_var'] = LpVariable(f"supply_{name}", lowBound=0, cat='Continuous')
                cont_count += 1
            else:
                node['supply_var'] = None

            if node['capacity'] < float('inf') and self.soft_constr:
                node['capacity_var'] = LpVariable(f"capacity_{name}", lowBound=0, cat='Continuous')
                cont_count += 1
            else:
                node['capacity_var'] = None

        if self.verbose: print(f"Variable types: {cont_count} continuous, {bin_count} binary")


    def _assign_objective_function(self):
        objective = 0
 
        # add edge costs
        for _, _, edge in super().edges.data():
            if edge['selection_var'] is not None: objective += edge['selection_var'] * edge['fixed_cost']
            if edge['flow_var'] is not None: objective += edge['flow_var'] * edge['variable_cost']
            if edge['capacity_var'] is not None: objective += edge['capacity_var'] * self.soft_penalty
        
        # add node costs
        for node_name, node in super().nodes.data():
            # add node selection costs
            if node['selection_var'] is not None: objective += node['selection_var'] * node['fixed_cost']
            # add node variable costs
            if node['variable_cost'] > 0:
                for _, _, edge in super().out_edges(node_name, data=True):
                    objective += edge['flow_var'] * node['variable_cost']
            
            # add soft constraint costs
            if node['supply_var'] is not None: objective += node['supply_var'] * self.soft_penalty
            if node['capacity_var'] is not None: objective += node['capacity_var'] * self.soft_penalty

        self.prob += objective, 'Objective',


    def _assign_constraints(self):
        # count of contraints
        constr_count = 0
        # add capacity constraints for edges with fixed costs
        for origin_name, destination_name, edge in super().edges.data():
            rhs = edge['capacity']
            if edge['selection_var'] is not None: rhs *= edge['selection_var']
            if edge['capacity_var'] is not None: rhs += edge['capacity_var']
            self.prob += edge['flow_var'] <= rhs, f"capacity_{origin_name}-{destination_name}",
            constr_count += 1
            
            # get origin node
            origin_node = super().nodes[origin_name]
            # check if origin node has a selection variable
            if origin_node['selection_var'] is not None:
                rhs = edge['capacity'] * origin_node['selection_var'] 
                if edge['capacity_var'] is not None: rhs += edge['capacity_var']
                self.prob += (edge['flow_var'] <= rhs, f"node_selection_{origin_name}-{destination_name}",)
                constr_count += 1

        total_demand = total_supply = 0
        # add flow conservation constraints
        for node_name, node in super().nodes.data():
            # aggregate in and out flows
            in_flow = 0
            for _, _, edge in super().in_edges(node_name, data=True):
                if edge['flow_var'] is not None: in_flow += edge['flow_var']
            
            out_flow = 0
            for _, _, edge in super().out_edges(node_name, data=True):
                if edge['flow_var'] is not None: out_flow += edge['flow_var']

            # add node capacity contraint
            if node['capacity'] < float('inf'):
                self.prob += out_flow <= node['capacity'], f"node_capacity_{node_name}",
                constr_count += 1

            # check what type of node it is
            if node['demand'] == node['supply']:
                # transshipment node: in_flow = out_flow
                self.prob += in_flow == out_flow, f"flow_balance_{node_name}",
            else:
                # in_flow - out_flow >= demand - supply - supply_added
                rhs = node['demand'] - node['supply']
                if node['supply_var'] is not None: rhs -= node['supply_var']
                self.prob += in_flow - out_flow >= rhs, f"flow_balance_{node_name}",
            constr_count += 1

            # update total demand and supply
            total_demand += node['demand']
            total_supply += node['supply']

        if self.verbose:
            print(f"Constraints: {constr_count}")
            print(f"Total supply: {total_supply}, Total demand: {total_demand}")
        

    def _assign_variable_values(self, opt_found):
        # assign decision variable values if optimal solution found, otherwise set to None
        # add utilisation, total fixed cost, total variable cost and total cost, inflow and outflow

        # assign edge values        
        for _, _, edge in super().edges.data():
            # initialize values
            edge['flow'] = edge['utilisation'] = edge['total_variable_cost'] = None
            edge['selected'] = edge['total_fixed_cost'] = edge['total_cost'] = None
            edge['capacity_added'] = edge['capacity_penalty_cost'] = None
            # check if optimal solution found
            if opt_found and edge['flow_var'] is not None:                    
                edge['flow'] = edge['flow_var'].varValue                    
                edge['total_variable_cost'] = edge['flow'] * edge['variable_cost']
                edge['total_cost'] = edge['total_variable_cost']
                capacity = edge['capacity']

                # check if capacity variable is used
                if edge['capacity_var'] is not None: 
                    edge['capacity_added'] = edge['capacity_var'].varValue 
                    capacity = edge['capacity'] + edge['capacity_added']
                    edge['capacity_penalty_cost'] = edge['capacity_added'] * self.soft_penalty
                    edge['total_cost'] += edge['capacity_penalty_cost']                        
                    capacity += edge['capacity_added']

                # calculate utilisation
                edge['utilisation'] = (edge['flow'] / capacity if 0 < capacity < float('inf') else None)

                if edge['selection_var'] is not None: 
                    edge['selected'] = edge['selection_var'].varValue
                    edge['total_fixed_cost'] = edge['selected'] * edge['fixed_cost']
                    edge['total_cost'] += edge['total_fixed_cost']

        # assign node values
        for node_name, node in super().nodes.data():
            # initialize values
            node['inflow'] = node['outflow'] = node['total_variable_cost'] = node['total_cost'] = node['utilisation'] = None
            node['selected'] = node['total_fixed_cost'] = node['total_cost'] = None
            node['capacity_added'] = node['supply_added'] = node['capacity_penalty_cost'] = node['supply_penalty_cost'] = None
            if opt_found:                
                # calculate inflow and outflow
                node['inflow'] = sum(edge['flow'] for _, _, edge in super().in_edges(node_name, data=True) 
                                     if edge['flow'] is not None)
                node['outflow'] = sum(edge['flow'] for _, _, edge in super().out_edges(node_name, data=True) 
                                      if edge['flow'] is not None)
                capacity = node['capacity']
                node['total_variable_cost'] = node['outflow'] * node['variable_cost']
                node['total_cost'] = node['total_variable_cost']
                
                # check if node has selection variable
                if node['selection_var'] is not None: 
                    node['selected'] = node['selection_var'].varValue 
                    node['total_fixed_cost'] = node['selected'] * node['fixed_cost']
                    node['total_cost'] += node['total_fixed_cost']

                # check if node as capacity variable
                if node['capacity_var'] is not None:
                    node['capacity_added'] = node['capacity_var'].varValue
                    capacity += node['capacity_added']
                    node['capacity_penalty_cost'] = node['capacity_added'] * self.soft_penalty
                    node['total_cost'] += node['capacity_penalty_cost']

                node['utilisation'] = (node['outflow'] / capacity if 0 < capacity < float('inf') else None)
                
                # check if node has supply variable
                if node['supply_var'] is not None:
                    node['supply_added'] = node['supply_var'].varValue
                    node['supply_penalty_cost'] = node['supply_added'] * self.soft_penalty
                    node['total_cost'] += node['supply_penalty_cost']
                    

    def _set_soft_penalty(self, soft_penalty=None):
        # set soft penalty
        if soft_penalty is not None:
            if soft_penalty <= 0: raise ValueError('soft_penalty must be positive')
            self.soft_penalty = soft_penalty
        else:
            # set soft penalty as total of all variable and fixed costs
            max_cost = 1
            max_cost += sum(edge['variable_cost'] + edge['fixed_cost'] for _, _, edge in super().edges.data())
            max_cost += sum(node['variable_cost'] + node['fixed_cost'] for _, node in super().nodes.data())
            self.soft_penalty = max_cost


    def min_cost_flow(self, soft_constr=False, soft_penalty=None, verbose=True):
        self.verbose = verbose
        self.soft_constr = soft_constr
        if self.verbose: print(f"Soft constraints: {soft_constr}")

        # get sinks with no path from any source
        no_path_sinks = self._get_no_path_sinks()
        if no_path_sinks:
            if self.verbose: print(f"Sinks with no path from any source: {no_path_sinks}")
            return 'Infeasible'

        # check if soft penalty needs to be set
        if soft_constr: 
            self._set_soft_penalty(soft_penalty)
            if self.verbose: print(f"Soft penalty: {self.soft_penalty:.2f}")

        start_time = time.time()
        # create LP problem
        self.prob = LpProblem("FlowGraph.min_cost_flow", LpMinimize)
        # assign decision variables
        self._assign_decision_variables()
        # assign objective function
        self._assign_objective_function()
        # assign constraints
        self._assign_constraints()
        if self.verbose: print(f"Model creation time: {time.time() - start_time:.2f} s")

        start_time = time.time()
        # solve LP problem
        self.prob.solve()
        solve_time = time.time() - start_time

        # get status
        status = LpStatus[self.prob.status]

        if status == 'Optimal':
            # get objective value
            objective = self.prob.objective.value()
            if self.verbose: print(f"Optimal solution found: {objective:.2f} in {solve_time:.2f} s")
        elif verbose:
            print(f"Optimization status: {status} in {solve_time:.2f} s")
        
        # assign variable values
        self._assign_variable_values(status=='Optimal')

        return status
    
    def _get_no_path_sinks(self):
        # get list of sinks with no path from any source

        # get sinks and sources
        sources, sinks = [], []
        for node_name, node in super().nodes(data=True):
            if node['demand'] > node['supply']: sinks.append(node_name)
            elif node['supply'] > node['demand']: sources.append(node_name)

        # check that there is at least one path from source to sink
        no_path_sinks = []
        for sink in sinks:    
            if not any(nx.has_path(self, source, sink) for source in sources): no_path_sinks.append(sink)
        
        return no_path_sinks    
    
    def infeasibility_analysis(self, verbose=True):
        # analyse why the model is infeasible
        # optimize with soft constraints
        self.min_cost_flow(soft_constr=True, verbose=verbose)

        # get nodes that require additional capacity of supply
        node_df = self.nodes_to_df()
        col_select = ['name', 'type', 'demand', 'supply', 'capacity_added', 'supply_added']
        infeasible_node_df = node_df.query('capacity_added > 0 or supply_added > 0')[col_select]

        # get edges that require additional capacity
        edge_df = self.edges_to_df()
        col_select = ['origin', 'destination', 'capacity', 'capacity_added']
        infeasible_edge_df = edge_df.query('capacity_added > 0')[col_select]


        if verbose and (not infeasible_node_df.empty or not infeasible_edge_df.empty):
            print("\nInfeasibility analysis:")

            for _, node in infeasible_node_df.iterrows():
                if node['capacity_added'] and node['capacity_added'] > 0: 
                    print(f"Node {node['name']} requires additional capacity: {node['capacity_added']}")
                if node['supply_added'] and node['supply_added'] > 0: 
                    print(f"Node {node['name']} requires additional supply: {node['supply_added']}")

            for _, edge in infeasible_edge_df.iterrows():
                if edge['capacity_added'] and edge['capacity_added'] > 0:
                    print(f"Edge {edge['origin']}-{edge['destination']} requires additional capacity: {edge['capacity_added']}")

        return {'nodes': infeasible_node_df, 'edges': infeasible_edge_df}

    
    def nodes_to_df(self):
        # columns to be dropped
        drop_cols = ['selection_var', 'supply_var', 'capacity_var']
        # convert nodes to dataframe
        if not self.soft_constr: drop_cols += ['supply_added', 'capacity_added', 'supply_penalty_cost', 
                                               'capacity_penalty_cost']
        df = (pd.DataFrame(dict(super().nodes(data=True))).T
              .reset_index()
              .rename(columns={'index': 'name'})
              .drop(columns=drop_cols, errors='ignore'))
        return df
    

    def edges_to_df(self):
        # columns to be dropped
        drop_cols = ['flow_var', 'selection_var', 'capacity_var']
        if not self.soft_constr: drop_cols += ['capacity_added', 'capacity_penalty_cost']
        # convert edges to dataframe
        df = (pd.DataFrame([{**{'origin': o, 'destination': d}, **attr} for o, d, attr in super().edges(data=True)])
              .drop(columns=drop_cols, errors='ignore'))
        return df


FlowGraph.Edge.update_forward_refs()


# Define Problem

- Nodes
    - Factories
        - number: 2
        - demand: -700
        - fixed cost: 100
        - location: rand
    - DCs
        - number: 4
        - fixed cost: 25
        - location: rand
    - Markets
        - number: 15
        - demand: rand 1-100
        - location: rand
- Edges
    - Factories - DCs
        - capacity: 350
        - var cost: distance
    - DCs - Markets
        - capacity: 100
        - var cost: distance

In [64]:
random.seed(115)
# Define nodes
factories = [FlowGraph.Node(name=f'Factory {i}', supply=700, type='Factory', fixed_cost=100, x=random.uniform(0, 2),
                             y=random.uniform(0, 1)) for i in range(2)]
dcs = [FlowGraph.Node(name=f'DC {i}', fixed_cost=25, capacity=500, type='DC', x=random.uniform(0, 2), 
                      y=random.uniform(0, 1)) for i in range(4)]
markets = [FlowGraph.Node(name=f'Market {i}', demand=random.randint(1, 100), type='Market', x=random.uniform(0, 2), 
                          y=random.uniform(0, 1)) for i in range(15)]

# Define edges
edges = []
# Factories to DCs
for factory in factories:
    for dc in dcs:
        distance = ((factory.x - dc.x)**2 + (factory.y - dc.y)**2)**0.5
        edges.append(FlowGraph.Edge(origin=factory, destination=dc, capacity=350, variable_cost=distance))

# DCs to Markets
for dc in dcs:
    for market in markets:
        distance = ((dc.x - market.x)**2 + (dc.y - market.y)**2)**0.5
        edges.append(FlowGraph.Edge(origin=dc, destination=market, capacity=100, variable_cost=distance))

# Create FlowGraph
G2 = FlowGraph(edges)
G2.viz.plot_graph().update_layout(height=600, width=1100).show()


In [65]:
# Create pos dictionary for drawing nodes at x, y coordinates
pos = {node.name: (node.x, node.y) for node in factories + dcs + markets}
type_color = {'Factory': 'red', 'DC': 'blue', 'Market': 'green'}
node_color = [type_color[node['type']] for _, node in G2.nodes(data=True)]

G2.min_cost_flow()

G2.viz.plot_flowgraph().update_layout(height=600, width=1100).show()


Soft constraints: False
Variable types: 68 continuous, 6 binary
Constraints: 161
Total supply: 1400.0, Total demand: 909.0
Model creation time: 0.01 s
Optimal solution found: 1334.88 in 0.07 s


# TODO

- Infeasibility analysis
    - connection
    - supply/demand increment
- plot network in plotly
- plot flow graph in plotly
- plot flows in plotly

# Infeasibility Analysis

In [66]:
# create infeasible graph
nodes = [FlowGraph.Node(name='A', supply=10), 
         FlowGraph.Node(name='B', demand=5), 
         FlowGraph.Node(name='C', demand=5),
         FlowGraph.Node(name='D', demand=20)]
edges = [FlowGraph.Edge(origin=nodes[0], destination=nodes[2], capacity=2), 
         FlowGraph.Edge(origin=nodes[0], destination=nodes[3], capacity=30), 
         FlowGraph.Edge(origin=nodes[0], destination=nodes[1], capacity=5)]
G3 = FlowGraph(nodes + edges)

In [67]:
G3.min_cost_flow(soft_constr=True)

Soft constraints: True
Soft penalty: 1.00
Variable types: 7 continuous, 0 binary
Constraints: 7
Total supply: 10.0, Total demand: 30.0
Model creation time: 0.00 s
Optimal solution found: 23.00 in 0.02 s


'Optimal'

In [68]:
G3.nodes_to_df()

,name,demand,supply,capacity,type,variable_cost,fixed_cost,x,y,inflow,outflow,total_variable_cost,total_cost,utilisation,selected,total_fixed_cost,capacity_added,supply_added,capacity_penalty_cost,supply_penalty_cost
0,A,0.0,10.0,inf,None,0.0,0.0,0.0,0.0,0,30.0,0.0,20.0,None,None,None,None,20.0,None,20.0
1,B,5.0,0.0,inf,NaN,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,5.0,0.0,inf,NaN,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,20.0,0.0,inf,NaN,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
infease_dict = G3.infeasibility_analysis(verbose=True)

Soft constraints: True
Soft penalty: 1.00
Variable types: 7 continuous, 0 binary
Constraints: 7
Total supply: 10.0, Total demand: 30.0
Model creation time: 0.00 s
Optimal solution found: 23.00 in 0.01 s

Infeasibility analysis:
Node A requires additional supply: 20.0
Edge A-C requires additional capacity: 3.0


# Plot

In [71]:
node_df = G2.nodes_to_df()
node_df.head()

,name,demand,supply,capacity,type,variable_cost,fixed_cost,x,y,inflow,outflow,total_variable_cost,total_cost,utilisation,selected,total_fixed_cost
0,Factory 0,0.0,700.0,inf,Factory,0.0,100.0,1.791805,0.196935,0,559.0,0.0,100.0,None,1.0,100.0
1,DC 0,0.0,0.0,500.0,DC,0.0,25.0,1.508608,0.858003,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DC 1,0.0,0.0,500.0,DC,0.0,25.0,1.98289,0.678457,177.0,177.0,0.0,25.0,0.354,1.0,25.0
3,DC 2,0.0,0.0,500.0,DC,0.0,25.0,1.280012,0.687317,350.0,350.0,0.0,25.0,0.7,1.0,25.0
4,DC 3,0.0,0.0,500.0,DC,0.0,25.0,0.965659,0.546996,382.0,382.0,0.0,25.0,0.764,1.0,25.0


In [72]:
edge_df = G2.edges_to_df()
edge_df.head()

,origin,destination,capacity,variable_cost,fixed_cost,flow,utilisation,total_variable_cost,selected,total_fixed_cost,total_cost
0,Factory 0,DC 0,350.0,0.719174,0.0,0.0,0.000000,0.000000,None,None,0.000000
1,Factory 0,DC 1,350.0,0.518052,0.0,177.0,0.505714,91.695117,None,None,91.695117
2,Factory 0,DC 2,350.0,0.708807,0.0,350.0,1.000000,248.082316,None,None,248.082316
3,Factory 0,DC 3,350.0,0.897251,0.0,32.0,0.091429,28.712033,None,None,28.712033
4,DC 0,Market 0,100.0,1.330406,0.0,0.0,0.000000,0.000000,None,None,0.000000


In [83]:

node_to_idx

{'Factory 0': 0,
 'DC 0': 1,
 'DC 1': 2,
 'DC 2': 3,
 'DC 3': 4,
 'Factory 1': 5,
 'Market 0': 6,
 'Market 1': 7,
 'Market 2': 8,
 'Market 3': 9,
 'Market 4': 10,
 'Market 5': 11,
 'Market 6': 12,
 'Market 7': 13,
 'Market 8': 14,
 'Market 9': 15,
 'Market 10': 16,
 'Market 11': 17,
 'Market 12': 18,
 'Market 13': 19,
 'Market 14': 20}

In [85]:
node_to_idx = {name: i for i, name in enumerate(node_df['name'])}
node_colors = G2.viz._get_color_dict(node_df['type'])

go.Figure(go.Sankey(
    node = dict(
        pad=15,
        thickness=20,
        label=node_df['name'],
        color=[node_colors[node] for node in node_df['type']],
        ),
    link=dict(
        source=[node_to_idx[origin] for origin in edge_df['origin']],
        target=[node_to_idx[destination] for destination in edge_df['destination']],
        value=edge_df['flow']
    )
)
).show()